In [18]:
import os
import subprocess
import numpy as np
import torch
from torch import nn
from utils_torch import * 

# Compilers and Interpreters

## Symbolic Programming

In [ ]:
def add_():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

In [2]:
print(evoke_())


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))


In [3]:
y = compile(evoke_(), '', 'exec')
exec(y)

10


## Hybridizing the Sequential Class

 replacing `Sequential` with `HybridSequential`. We begin by defining a simple MLP.

In [4]:
# Factory for networks
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[ 0.0961, -0.1060]], grad_fn=<AddmmBackward0>)

By converting the model using torch.jit.script function, we are able to compile and optimize the computation in the MLP. The model’s computation result remains unchanged.

In [5]:
net = torch.jit.script(net)
net(x)

tensor([[ 0.0961, -0.1060]], grad_fn=<AddmmBackward0>)

## Acceleration by Hybridization

In [8]:
class Benchmark:
    def __init__(self, description='Done'):
        self.description = description
        
    def __enter__(self):
        self.timer=Timer()
        return self
    
    def __exit__(self, *args):
        print(f"{self.description}:{self.timer.stop():.4f} sec")

In [11]:
net = get_net()
with Benchmark('Without torchscript'):
    for i in range(1000):
        net(x)

Without torchscript:0.0653 sec


In [12]:
net = torch.jit.script(net)
with Benchmark('With torchscript'):
    for i in range(1000):
        net(x)

With torchscript:0.0516 sec


As is observed in the above results, after an nn.Sequential instance is scripted using the `torch.jit.script` function, computing performance is improved through the use of symbolic programming.

### Serialization

 serialize (save) the model and its parameters to disk

In [13]:
net.save('my_mlp')

# Asynchronous Computation

##  Asynchrony via Backend

In [19]:
device = try_gpu()
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with Benchmark('numpy'):
    for _ in range(10):
        a = np.random.normal(size=(1000, 1000))
        b = np.dot(a, a)

with Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy:0.3322 sec
torch:0.0003 sec


The benchmark output via PyTorch is orders of magnitude faster. Reasons:
1. NumPy dot product is executed on the CPU processor while PyTorch matrix multiplication is executed on GPU and hence the latter is expected to be much faster. 
2. By default, GPU operations are asynchronous in PyTorch. Forcing PyTorch to finish all computation prior to returning shows what happened previously: computation is being executed by the backend while the frontend returns control to Python.

In [ ]:
np.random